1) необходимо установить все библиотеки

2) изменить путь для XML файла , код находится почти в самом конце(generator(elem, '.../.xml')

3) изменить путь для папки с фотографиями (for i in glob.glob('.../*.jpg')

4) запустить, должно работать (на выходе получается XML файл)

5) вручную нужно будет сделать только шапку для XML (то тега <image ...) 

In [1]:
# import necessary libraries
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
import torchvision
import torch
import numpy as np
import cv2
import random
import os
import glob
from tabulate import tabulate
import requests
import glob
import pandas as pd

# get the pretrained model from torchvision.models
# Note: pretrained=True will get the pretrained weights for the model.
# model.eval() to use the model for inference
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()


COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]
#person - 0 \\\ car - 1
threshold = 0.9
id_pic = 0
id_phh = 0

cv_img = []
predict = []
photo = []
id_ph = []
rez_to_txt = np.array
for i in glob.glob('/Users/awesome/Desktop/programming/ML/CyberControl/*.jpg'):
    img = Image.open(i)
    photo_name = img.filename
    transform = T.Compose([T.ToTensor()])
    img = transform(img)
    id_phh += 1
    pred = model([img])
    pred_score = list(pred[0]['scores'].detach().numpy())
    pred_t = [pred_score.index(x) for x in pred_score if x>threshold][-1]
    pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())]
    pred_boxes = ([(i[0], i[1], i[2], i[3]) for i in list(pred[0]['boxes'].detach().numpy())])
    #pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())]
    pred_boxes = pred_boxes[:pred_t+1]
    pred_class = pred_class[:pred_t+1]
       
    cv_img.append(img)
    predict.append(pred_boxes)
    predict.append(pred_class)
    photo.append(photo_name)
    id_ph.append(id_phh)
    
    id_pic = id_pic + 1
    rez_to_xml = np.array(predict)
    #df = pd.DataFrame(data=numpy_data, index=["row1", "row2"], columns=["column1", "column2"])
    #rez_to_xml = rez_to_xml.transpose()
    #rez_to_xml

   
#работает норм, бокс + класс запихивает в массив, надо сделать так,
#чтобы после выполнения каждого цикла шла запись в тхт файл (РАЗНЫЙ)
#color

In [2]:
# костыльные преобразования в нужный формат
rez_to_xml = rez_to_xml.tolist()
ttt = tuple()
rez_arr=()

In [3]:
a=0
b=2
for i in range(len(photo)):
    t_t = list(ttt)
    t_t.append(rez_to_xml[a:b])
    #print(rez_to_xml[a:b])
    a+=2
    b+=2
    t_t = tuple(t_t)
    rez_arr += t_t  

In [4]:
photo_name = 0
def generator(array, filename, photo, id_ph, photo_name):
    with open(filename, 'a+') as file:     
        name = f'<image id="{id_ph[photo_name]}" name="{photo[photo_name]}" width="2560" height="1440">'

            
        for index in range(len(array[0])):
            name += f'\n\t<box label="{array[1][index]}" occluded="0" xtl="{array[0][index][0]}" ytl="{array[0][index][1]}" ' \
                    f'xbr="{array[0][index][2]}" ybr="{array[0][index][3]}">\n\t</box>'

        name += '\n</image>\n'
        file.writelines(name)


if __name__ == '__main__':
    for elem in rez_arr:
        generator(elem, '/Users/awesome/Desktop/programming/ML/CyberControl/итоговый.xml', photo, id_ph, photo_name)
        photo_name = photo_name + 1